In [1]:
#%matplotlib notebook
%matplotlib widget
from zMag_Field import *
import numpy as np
import matplotlib.animation as animation


<IPython.core.display.Javascript object>

In [2]:
x_current = 5
y_current = 5
position = (0.0, 0.0, 0.03)
coil_pillars = construct_system(x_current, y_current)
dBx_dx, dBy_dy, dBz_dz = compute_magnetic_gradient_xyz(coil_pillars, position, delta=0.001)
magnetic_moment = compute_magnetic_moment()
magnetic_force_x = dBx_dx * magnetic_moment
magnetic_force_y = dBy_dy * magnetic_moment
magnetic_force_z = dBz_dz * magnetic_moment

print("Magnetic force in x direction: ", magnetic_force_x)
print("Magnetic force in y direction: ", magnetic_force_y)
print("Magnetic force in z direction: ", magnetic_force_z)

Magnetic force in x direction:  1.5143717581141966e-16
Magnetic force in y direction:  0.0
Magnetic force in z direction:  -8.191250482628502e-17


In [3]:
x_current = -5
y_current = -5
position = (0.0, 0.0, 0.03)
coil_pillars = construct_system(x_current, y_current)
dBx_dx, dBy_dy, dBz_dz = compute_magnetic_gradient_xyz(coil_pillars, position, delta=0.001)
magnetic_moment = compute_magnetic_moment()
magnetic_force_x = dBx_dx * magnetic_moment
magnetic_force_y = dBy_dy * magnetic_moment
magnetic_force_z = dBz_dz * magnetic_moment

print("Magnetic force in x direction: ", magnetic_force_x)
print("Magnetic force in y direction: ", magnetic_force_y)
print("Magnetic force in z direction: ", magnetic_force_z)

Magnetic force in x direction:  -1.5143717581141966e-16
Magnetic force in y direction:  0.0
Magnetic force in z direction:  8.191250482628502e-17


In [4]:
x_current = -5
y_current = 5
position = (0.0, 0.0, 0.03)
coil_pillars = construct_system(x_current, y_current)
dBx_dx, dBy_dy, dBz_dz = compute_magnetic_gradient_xyz(coil_pillars, position, delta=0.001)
magnetic_moment = compute_magnetic_moment()
magnetic_force_x = dBx_dx * magnetic_moment
magnetic_force_y = dBy_dy * magnetic_moment
magnetic_force_z = dBz_dz * magnetic_moment

print("Magnetic force in x direction: ", magnetic_force_x)
print("Magnetic force in y direction: ", magnetic_force_y)
print("Magnetic force in z direction: ", magnetic_force_z)

Magnetic force in x direction:  0.0
Magnetic force in y direction:  1.448066832488656e-16
Magnetic force in z direction:  1.5880438976981305e-17


In [5]:
x_current = 5
y_current = -5
position = (0.0, 0.0, 0.03)
coil_pillars = construct_system(x_current, y_current)
dBx_dx, dBy_dy, dBz_dz = compute_magnetic_gradient_xyz(coil_pillars, position, delta=0.001)
magnetic_moment = compute_magnetic_moment()
magnetic_force_x = dBx_dx * magnetic_moment
magnetic_force_y = dBy_dy * magnetic_moment
magnetic_force_z = dBz_dz * magnetic_moment

print("Magnetic force in x direction: ", magnetic_force_x)
print("Magnetic force in y direction: ", magnetic_force_y)
print("Magnetic force in z direction: ", magnetic_force_z)

Magnetic force in x direction:  0.0
Magnetic force in y direction:  -1.448066832488656e-16
Magnetic force in z direction:  -1.5880438976981305e-17


In [ ]:
force_vector = (magnetic_force_x, magnetic_force_y, magnetic_force_z)
new_x, new_y, new_z = update_position(position, force_vector, mass=0.025, dt=0.025)
print("New position: ", new_x, new_y, new_z)
new_x, new_y, new_z = update_position(position, force_vector, mass=0.025, dt=1)
print("New position: ", new_x, new_y, new_z)



New position:  0.0 -1.8100835406108202e-18 0.03
New position:  0.0 -2.8961336649773117e-15 0.02999999999999968


In [17]:
len(coil_pillars)
print(coil_pillars)

[[Circle(id=1902658937296), Circle(id=1902658940880), Circle(id=1902658934416), Circle(id=1902658941392), Circle(id=1902658936720), Circle(id=1902658941648), Circle(id=1902658941136), Circle(id=1902658941328), Circle(id=1902658941776), Circle(id=1902658941200), Circle(id=1902658942032), Circle(id=1902658941904), Circle(id=1902658941840), Circle(id=1902658942096), Circle(id=1902658942224), Circle(id=1902658942352), Circle(id=1902658942480), Circle(id=1902658942608), Circle(id=1902658942736), Circle(id=1902658942864), Circle(id=1902658942992), Circle(id=1902658943120), Circle(id=1902658943248), Circle(id=1902658943376), Circle(id=1902658943504), Circle(id=1902658938320), Circle(id=1902658943824), Circle(id=1902658943952), Circle(id=1902658944080), Circle(id=1902658944208)], [Circle(id=1902658944336), Circle(id=1902658944464), Circle(id=1902658944656), Circle(id=1902658944784), Circle(id=1902658944912), Circle(id=1902658938128), Circle(id=1902654239568), Circle(id=1902654238096), Circle(i

In [19]:
import numpy as np

def compute_magnetic_force(coil_pillars, position, magnetic_moment, delta=0.001):
    """
    Computes the magnetic force on a dipole at a given position.

    Parameters:
        coil_pillars (list): Coil definitions.
        position (tuple): (x, y, z) coordinates.
        magnetic_moment (tuple): Dipole moment vector (mx, my, mz) in A·m².
        delta (float): Small step for numerical differentiation.

    Returns:
        tuple: Force vector (Fx, Fy, Fz) in Newtons.
    """
    x, y, z = position
    mx, my, mz = magnetic_moment

    def get_B(pos):
        return compute_magnetic_field(coil_pillars, pos)

    # Compute gradient tensor G[i][j] = ∂Bi/∂xj
    G = [[0.0]*3 for _ in range(3)]
    for j, (dx, dy, dz) in enumerate([(delta,0,0), (0,delta,0), (0,0,delta)]):
        B_forward = get_B((x + dx, y + dy, z + dz))
        B_backward = get_B((x - dx, y - dy, z - dz))
        for i in range(3):
            G[i][j] = (B_forward[i] - B_backward[i]) / (2 * delta)

    # Compute force = G ⋅ m
    m_vec = np.array([mx, my, mz])
    G_matrix = np.array(G)
    F = G_matrix @ m_vec

    return tuple(F)

magnetic_moment = (0.0, 0.0, 2.06)
F = compute_magnetic_force(coil_pillars, position, magnetic_moment)


In [22]:
def update_position_velocity(position, velocity, force, mass, dt= 0.005):
    """
    Updates position and velocity using force and time step (basic Newtonian mechanics).
    
    Parameters:
        position (tuple): Current (x, y, z) position.
        velocity (tuple): Current (vx, vy, vz) velocity.
        force (tuple): Force vector (Fx, Fy, Fz).
        mass (float): Mass of the object.
        dt (float): Time step.
    
    Returns:
        tuple: Updated (position, velocity)
    """
    x, y, z = position
    vx, vy, vz = velocity
    Fx, Fy, Fz = force

    # Acceleration
    ax = Fx / mass
    ay = Fy / mass
    az = Fz / mass

    # Update velocity
    vx_new = vx + ax * dt
    vy_new = vy + ay * dt
    vz_new = vz + az * dt

    # Update position
    x_new = x + vx * dt + 0.5 * ax * dt**2
    y_new = y + vy * dt + 0.5 * ay * dt**2
    z_new = z + vz * dt + 0.5 * az * dt**2

    return (x_new, y_new, z_new), (vx_new, vy_new, vz_new)


In [24]:
p, v = update_position_velocity(position, (0, 0, 0), F, 0.025)
print(p)
print(v)

(np.float64(-0.00034177873676029323), np.float64(4.264069735596555e-20), np.float64(0.03))
(np.float64(-0.13671149470411728), np.float64(1.705627894238622e-17), np.float64(-3.173514816006643e-18))


In [ ]:
class PID:
    def __init__(self, kp, ki, kd, setpoint=0.0, output_limits=(-np.inf, np.inf), dt=0.005):
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.setpoint = setpoint
        self.output_limits = output_limits
        self.dt = dt
        self._integral = 0
        self._prev_error = 0

    def compute(self, measurement):
        error = self.setpoint - measurement
        self._integral += error * self.dt
        derivative = (error - self._prev_error) / self.dt
        output = self.kp * error + self.ki * self._integral + self.kd * derivative
        self._prev_error = error
        return max(self.output_limits[0], min(self.output_limits[1], output))


In [ ]:
pid_x = PID(kp=0.6, ki=0.0, kd=0.01, setpoint=0.0, output_limits=(-2.0, 2.0))
pid_y = PID(kp=0.6, ki=0.0, kd=0.01, setpoint=0.0, output_limits=(-2.0, 2.0))

position = np.array([0.01, -0.01, 0.03])  # Starting position of the magnet
velocity = np.array([0.0, 0.0, 0.0])  # Starting velocity of the magnet
mass = 0.025  # kg
dt = 0.005  # seconds
trajectory = []

# Assume initial coil currents (can start at 0)
current_x = 0.0
current_y = 0.0

for step in range(1000):
    x, y, z = position

    # 1. PID computes desired current adjustment
    delta_Ix = pid_x.compute(x)
    delta_Iy = pid_y.compute(y)

    # 2. Update the actual coil currents
    current_x += delta_Ix
    current_y += delta_Iy

    # 3. Update the current in the coils
    coil_pillars = construct_system(x_current, y_current)

    # 4. Compute magnetic force at current position
    force = compute_magnetic_force(coil_pillars, position)

    # 5. Zero out Z if we're assuming it's fixed
    force[2] = 0.0

    # 6. Update position from force
    position, velocity = update_position_velocity(position, velocity , force, mass, dt)
    
    # 7. Save trajectory
    trajectory.append(position.copy())
